In [1]:
import os
import sys
import warnings
import time
from pynq import PL
from pynq import Overlay


BF = os.path.join("./data/system_wrapper.bit")



if not os.path.exists(BF):
    print ("%s Doesn't exist!" % BF)
else:
    print ("Found bit file!")
    
    ol = Overlay(BF)
    ol.download()

    print ("Bitfile downloaded!")



Found bit file!
Bitfile downloaded!


In [5]:
from pynq.lib.video import *
from pynq.lib.video.hierarchies import *
from drivers.video_mixer import VideoMixer
from drivers.dynamic_clock import DynamicClock
from drivers.timing_controller import TimingController
from drivers.test_pattern_generator import TestPatternGenerator



                
print ("Starting Video")

print ("Configuring Timing Controller")
tc = TimingController(ol.ip_dict["video/timing_generator"])
tc.reset()
while (not tc.is_reset_done()):
    print (".")

WIDTH   = tc.get_generator_width()
HEIGHT  = tc.get_generator_height()
print ("Image Size (Retrieved from Timing Controller): %d x %d" % (WIDTH, HEIGHT))
print ("Configuring Test Pattern Generator")
tpg = TestPatternGenerator(ol.ip_dict["video/v_tpg_0"], debug = False)
tpg.set_image_size(WIDTH, HEIGHT)
tpg.set_color_format_to_rgb()
tpg.set_color_bar_test_pattern()
tpg.start()

print ("Configuring Video Mixer")
vm = VideoMixer(ol.ip_dict["video/v_mix_0"], WIDTH, HEIGHT)
vm.configure_layer(0, 0, 0, WIDTH, HEIGHT)
vm.configure_master_layer(WIDTH, HEIGHT)
#vm.configure_layer(1, 0, 0, WIDTH, HEIGHT)
#vm.enable_overlay_layer(True)

#print ("Video Mixer Control Register Before Enable: 0x%08X" % vm.get_control())
vm.start()

#print ("Video Mixer Control Register After Enable: 0x%08X" % vm.get_control())
print ("Enable Master Layer (Test Pattern) Output")
vm.enable_master_layer(True)
#print ("VM Settings:")
#print ("  x pos:   %d" % vm.get_layer_x(0))
#print ("  y pos:   %d" % vm.get_layer_y(0))
#print ("  width:   %d" % vm.get_layer_width(0))
#print ("  height:  %d" % vm.get_layer_height(0))
#print ("  scale:   %d" % vm.get_layer_scale(0))
#print ("  alpha:   %d" % vm.get_layer_alpha(0))
#print ("  stride:  %d" % vm.get_layer_stride(0))
#print ("  p1 Addr: %d" % vm.get_layer_plane1_addr(0))
#print ("  p2 Addr: %d" % vm.get_layer_plane2_addr(0))

#print ("Layer Settings: 0x%08X" % vm.get_layer_enable_reg())

g0 = ol.low_speed.axi_gpio_0
g0.write(0x0C, 0x01)
tc.enable(gen_enable = True, use_gen_src = True)
#print ("TC Control Register: 0x%08X" % tc.get_control_reg())



Starting Video
Configuring Timing Controller
Image Size (Retrieved from Timing Controller): 1920 x 1080
Configuring Test Pattern Generator
Configuring Video Mixer
Enable Master Layer (Test Pattern) Output
